In [1]:
import time
import torch
import cv2
from models.common import DetectMultiBackend
# from utils.dataloaders import LoadImages
from utils.general import Profile, check_img_size, non_max_suppression, scale_boxes
from utils.torch_utils import select_device
# from ultralytics.utils.plotting import Annotator

# __file__ = '/home/nnthao/project/yolov5/detect.py'
# FILE = Path(__file__).resolve()
# ROOT = FILE.parents[0]  # YOLOv5 root directory
# if str(ROOT) not in sys.path:
#     sys.path.append(str(ROOT))  # add ROOT to PATH
# ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

/media/nnthao/miniconda3/envs/mat_inpainting/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def dmm_tensor(t, p=''):
    if type(t) == list:
        print(p + 'list', len(t))
        for i in t:
            dmm_tensor(i, p + '  ')
    else:
        print(p + 'tensor', t.shape)

In [3]:
weights = '/media/nnthao/yolov5/runs/train/exp-mask/weights/best.pt'    # model path or triton URL
data = '/home/nnthao/project/yolov5/data/datamask.yaml' # dataset.yaml path
device = 'cpu'  # cuda device, i.e. 0 or 0,1,2,3 or cpu
half=False  # use FP16 half-precision inference
dnn=False   # use OpenCV DNN for ONNX inference
imgsz = (512, 512)  # inference size (height, width)

# Load model
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data)
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size

YOLOv5 🚀 v7.0-249-gf400bba Python-3.8.18 torch-1.8.0+cu111 CPU

Fusing layers... 
Model summary: 267 layers, 46189053 parameters, 0 gradients, 107.9 GFLOPs


In [4]:
# source = '/home/nnthao/project/FDA/test_sets/CelebA-HQ/masked_images/test1.png' # file/dir/URL/glob/screen/0(webcam)
# vid_stride=1    # video frame-rate stride

# # Dataloader
# batch_size = 1
# dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
# vid_path, vid_writer = [None] * batch_size, [None] * batch_size

In [88]:
augment=False   # augmented inference
# save_dir = 'runs/detect/exp'
visualize=False # visualize features
conf_thres=0.25 # confidence threshold
iou_thres=0.45  # NMS IOU threshold
classes=None    # filter by class: --class 0, or --class 0 2 3
agnostic_nms=False  # class-agnostic NMS
max_det=18  # maximum detections per image

batch_size = 1  # batch_size

# line_thickness=1  # bounding box thickness (pixels)
box_color = (255, 255, 255)

# Run inference
model.warmup(imgsz=(1 if pt or model.triton else batch_size, 3, *imgsz))  # warmup
dt = (Profile(), Profile(), Profile())

from torchvision.io import read_image
images_in = ((torch.cat((read_image('/home/nnthao/project/FDA/test_sets/CelebA-HQ/images/test1.png')[None, ...], 
       read_image('/home/nnthao/project/FDA/test_sets/CelebA-HQ/images/test2.png')[None, ...]))) / 255).to(device)

# for im in images_in:
# for path, im, im0s, vid_cap, s in dataset:
with dt[0]:
    # images_in = torch.from_numpy(images_in).to(model.device)
    images_in = images_in.half() if model.fp16 else images_in.float()  # uint8 to fp16/32
    # images_in /= 255  # 0 - 255 to 0.0 - 1.0
    if len(images_in.shape) == 3:
        images_in = images_in[None]  # expand for batch dim

# Inference
with dt[1]:
    # visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    infer_st = time.time()
    pred = model(images_in, augment=augment, visualize=visualize)
    infer_t = time.time() - infer_st

    print('infer time: ', infer_t)

# NMS
with dt[2]:
    nms_st = time.time()
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    nms_t = time.time() - nms_st

    print('nms time: ', nms_t)

tensor_trans_st = time.time()

# >>> use sefl-code >>>
# im0 = torch.zeros_like(im[0][0])

# for coor in pred[0][..., :-2].floor().int():
#     for y in range(512):
#         for x in range(512):
#             if x < coor[2] and x > coor[0] and y < coor[3] and y > coor[1]:
#                 im0[y][x] = 1
# <<< use sefl-code <<<

# >>> use Annotator >>>
# for i, det in enumerate(pred):  # per image
#     im0 = (im.permute(2, 3, 1, 0).reshape(512, 512, 3) * 255).floor().contiguous().cpu().detach().numpy()
#     annotator = Annotator(im0, line_width=line_thickness)

#     if len(det):
#         # Rescale boxes from img_size to im0 size
#         det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

#         # Write results
#         for *xyxy, conf, cls in reversed(det):
#             annotator.box_label(xyxy, None, color=box_color)

#     im0 = annotator.result()
# <<< use Annotator <<<

# >>> use cv2 >>>
dets_in = None
for i, det in enumerate(pred):  # per image
    det_in = torch.zeros(512, 512, 3, dtype=torch.uint8).contiguous().cpu().detach().numpy()

    if len(det):
        # Rescale boxes from img_size to det_in size
        det[:, :4] = scale_boxes(images_in.shape[2:], det[:, :4], det_in.shape).round()

        # Write results
        for *xyxy, conf, cls in reversed(det):
            cv2.rectangle(det_in, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), box_color, -1)

    det_in = torch.from_numpy(det_in[..., 0].reshape(1, 1, 512, 512)).to(model.device)
    det_in = det_in.half() if model.fp16 else det_in.float()  # uint8 to fp16/32
    det_in /= 255  # 0 - 255 to 0.0 - 1.0

    if i:
        dets_in = torch.cat((dets_in, det_in))
    else:
        dets_in = det_in
# <<< use cv2 <<<

tensor_trans_t = time.time() - tensor_trans_st
print('tensor transform time: ', tensor_trans_t)

infer time:  10.72494649887085
nms time:  0.004666566848754883
tensor transform time:  0.18992400169372559


In [93]:
for det_in in dets_in:
    print(det_in.shape)

torch.Size([1, 512, 512])
torch.Size([1, 512, 512])


In [94]:
import matplotlib.image as mpimg
import numpy as np

for det_in in dets_in:
    mpimg.imsave('./imshow-ngu-vc.png', np.concatenate(((det_in.permute(1, 2, 0) * 255).floor().contiguous().cpu().detach().numpy(), 
                                                (det_in.permute(1, 2, 0) * 255).floor().contiguous().cpu().detach().numpy(), 
                                                (det_in.permute(1, 2, 0) * 255).floor().contiguous().cpu().detach().numpy()), axis=2).astype('uint8'))
    input()